In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Reshape, Conv1D, Conv2D, Add, BatchNormalization, MaxPooling1D)

In [2]:

#embedding
myInput = Input(shape=(20,1912))
reshaped = Reshape(target_shape=(20,1912,1))(myInput)
myConv2d = Conv2D(filters=256, kernel_size=(20,1))(reshaped)
reshaped = Reshape(target_shape=(1912,256))(myConv2d)
normalized = BatchNormalization()(reshaped)


model = Model(myInput, normalized)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 1912)          0         
_________________________________________________________________
reshape (Reshape)            (None, 20, 1912, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1, 1912, 256)      5376      
_________________________________________________________________
reshape_1 (Reshape)          (None, 1912, 256)         0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 1912, 256)         1024      
Total params: 6,400
Trainable params: 5,888
Non-trainable params: 512
_________________________________________________________________


In [3]:
#residual layer
aconv1d = Conv1D(filters=256, kernel_size=3)(normalized)
bconv1d = Conv1D(filters=256, kernel_size=3)(aconv1d)
cconv1d = Conv1D(filters=256, kernel_size=1)(bconv1d)
normalized = BatchNormalization()(cconv1d)

model = Model(myInput, normalized)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 1912)          0         
_________________________________________________________________
reshape (Reshape)            (None, 20, 1912, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1, 1912, 256)      5376      
_________________________________________________________________
reshape_1 (Reshape)          (None, 1912, 256)         0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 1912, 256)         1024      
_________________________________________________________________
conv1d (Conv1D)              (None, 1910, 256)         196864    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1908, 256)         196864    
__________

In [6]:
#residual layer + pooling
raconv1d = Conv1D(filters=256, kernel_size=3)(normalized)
lconv1d = Conv1D(filters=256, kernel_size=3)(normalized)
rbconv1d = Conv1D(filters=256, kernel_size=3)(aconv1d)
rcconv1d = Conv1D(filters=256, kernel_size=1)(bconv1d)
rmaxpooling = MaxPooling1D(pool_size=2)(rcconv1d)
lmaxpooling = MaxPooling1D(pool_size=2)(lconv1d)
add = Add()([rmaxpooling, lmaxpooling])


model = Model(myInput, add)
model.summary()

ValueError: Operands could not be broadcast together with shapes (954, 256) (953, 256)